In [1]:
#####PROGRAM TO SIZE AND EVALUATE THE  PERFORMANCE OF RAINWATER HARVESTING (RWH) SYSTEMS
##Input data: sizing and economic parameters file, DUs files (tariff, costs and discount rate), rainfall file and demand file.
##Obs: the results files are created by the program and saved to the folder where that script is saved.

import csv, math
from decimal import Decimal
from datetime import date, datetime, timedelta

#OPEN INPUT FILES 
ne=('inputcode3_IP_demand3_area200') #input file name with sizing and economic parameters
with open (ne+'.csv', 'r') as file:
    file1=csv.reader(file,delimiter=';') #read the file delimited by ";"
    rows=list(file1) #turn the file into a list
file.close()

##Sizing data
area=Decimal(rows[4][0]) #catchment area
ces=Decimal(rows[4][1]) #surface runoff coef
discard_ini=Decimal(rows[4][2]) #initial discard
teta=Decimal(rows[8][0]) #theta to define whether YBS or YAS
efficiency=Decimal(rows[10][0]) #desired efficiency
date0=datetime.strptime(rows[12][0],'%d/%m/%Y') #initial date

##Economic evaluation data 
###make a list of reservoir volumes and another with the costs
volumes=Decimal(rows[35][1]) #volume quantity 
qtvolume=0 #volume adder
l_volumes=[] #reservoir volumes
l_costsres=[] #cost per volume
while qtvolume<volumes: #while the counter is less than the considered qty
    l_volumes.append(Decimal(rows[36][qtvolume])) #makes a list with the volumes of the reservoirs
    l_costsres.append(Decimal(rows[37][qtvolume])) #make a list with the cost per volume
    qtvolume+=1
    
cmonthlyf=Decimal(rows[18][0]) #fixed monthly cost
cmonthlyv=Decimal(rows[18][1]) #variable monthly cost with the rainwater consumed volume
cquarterly=Decimal(rows[18][2]) #fixed quarterly cost 
csemester=Decimal(rows[18][3]) #fixed semester cost
cannualf=Decimal(rows[18][4]) #fixed annual cost
cannualv=Decimal(rows[18][5]) #variable annual cost with the initial investment
lifespan=Decimal(rows[22][0]) #lifespan
td=Decimal(rows[22][1]) #discount rate (% per month)
esg=Decimal(rows[22][3]) #percentage of sewage (%)
tmi=rows[26][1] #whether a minimum tariff is charged (y or n) 
if tmi.upper()=='Y': #if there is a minimum tariff, the program reads its value
    tmin=Decimal(rows[26][2])
tbas=rows[26][3] #whether there is a basic tariff (y or n)
if tbas.upper()=='Y': #if there is a basic tariff, the program reads its value
    tbasic=Decimal(rows[26][4])
else:
    tbasic=0
###make a list of consumption ranges and another with the respective tariffs
consumption_range=Decimal(rows[26][0]) #number of consumption ranges
fc=0 #consumption ranges counter
l_v=[] #list of volumes by consumption ranges
l_t=[] #list of tariffs by consumption ranges
while fc<consumption_range*2:
    l_v.append(Decimal(rows[28][fc])) #add the elements to the consumption range volume list
    l_t.append(Decimal(rows[28][fc+1])) #add the elements to the tariff list
    fc+=2
###create the average monthly consumption list 
l_c=[] #list of average monthly consumption for the building
x=0 #monthly consumption counter
while x<12:
    l_c.append(Decimal(rows[32][x])) #add the elements in the monthly consumption list
    x+=1

##Input data uncertainties
with open ('inputcode3_Tariff_cte.csv', 'r') as file2: #open the file and turn it into a list
    file2=csv.reader(file2,delimiter=',')
    l_tariff=list(file2)
with open ('inputcode3_Costs_cte.csv', 'r') as file3: #open the file and turn it into a list
    file3=csv.reader(file3,delimiter=',')
    l_costs=list(file3)
with open ('inputcode3_Discountrate_cte.csv', 'r') as file4: #open the file and turn it into a list
    file4=csv.reader(file4,delimiter=',')
    l_discountrate=list(file4) 

##Open precipitation and demand files and transform files to list
###Open precipitation file
file_prec=('inputcode3_Rain IP 1000 years') #rainfall file name (data in mm) - the number of years must be the same than in each scenario 
l_precipitation=[] #list precipitation data
with open(file_prec+'.csv', newline='') as file2: #open the file and turn each line into a list
    file=csv.reader(file2,delimiter=';')  
    for row2 in file:
        l_precipitation.append(row2)
lp=len(l_precipitation) #amount of elements in the precipitation list
###Open demand file
file_demand=('inputcode3_Demand3_IP') #demand file name (data in liters)
file3=open(file_demand+'.csv', 'r') #demand data list
d0=[]
for row3 in file3: #transform the file into a list
    d0.append(Decimal(row3))
file3.close()
ld=len(d0) #number of elements in the demand list
###Calculate precipitation series size in years
enddate=date0+timedelta(days=lp) #end date =start + series size     
qtdays=enddate-date0 # qty of days
years=Decimal((qtdays.days)/365) #days to years

#OPEN RESULTS FILE
with open ('results_IP_demand3_area200.csv','w') as file: #first line of the results file
    file.write('SCENARIO'+','+'Rainfall (mm/year)'+','+'Rate increase tariff tot (%/year)'+','+'Rate increase costs tot (%/year)'+\
                  ','+'Rate increase discount tot (%/year)'+','+'Area (m²)'+','+'Demand non-potable (m³/year)'+\
                  ','+'V ideal (m³)'+','+'Satis (%)'+','+'Consumption (m³/year)'+','+'Vextra  (m³/year)'+','+'NPV'+\
                  ','+'NPVV'+','+'BCR'+','+'REL'+','+'RH (%)'+','+'Initial cost'+\
                  ','+'V maxnpv (m³)'+','+'Satis npv (%)'+','+'Cons NPV (m³/year)'+','+'Vextra NPV (m³/year)'+','+'NPV max'+\
                  ','+'NPVV NPV'+','+'BCR NPV'+','+'REL NPV'+','+'RH NPV (%)'+\
                  ','+'V maxBCR (m³)'+','+'Satis BCR (%)'+','+'Cons BCR  (m³/year)'+','+'Vextra BCR  (m³/year)'+','+'NPV BCR'+\
                  ','+'NPVV BCR'+','+'BCR max'+','+'REL BCR'+','+'RH BCR (%)'+\
                  ','+'V maxNPVV (m³)'+','+'Satis NPVV (%)'+','+'Cons NPVV (m³/year)'+','+'Vextra NPVV (m³/year)'+','+'NPV NPVV'+\
                  ','+'NPVV max'+','+'BCR NPVV'+','+'rel NPVV'+','+'RH NPVV (%)'+\
                  ','+'V maxREL (m³)'+','+'Satis REL (%)'+','+'Cons REL  (m³/year)'+','+'Vextra REL  (m³/year)'+','+'NPV REL'+\
                  ','+'NPVV REL'+','+'BCR REL'+','+'REL max'+','+'RH REL (%)'+\
                  ','+'V maxsatis (m³)'+','+'Satis max (%)'+','+'Cons satis  (m³/year)'+','+'Vextra satis  (m³/year)'+','+'NPV satis'+\
                  ','+'NPVV satis'+','+'BCR satis'+','+'REL satis'+','+'RH satis (%)'+\
                  ','+'V maxRH (m³)'+','+'Satis RH (%)'+','+'Cons RH  (m³/year)'+','+'Vextra RH  (m³/year)'+','+'NPV RH'+\
                  ','+'NPVV RH'+','+'BCR RH'+','+'REL RH'+','+'RH max (%)'+'\n')
#SIZING
scenar=0
for tariff, costs, rdiscount, precipitation in zip (l_tariff, l_costs, l_discountrate, l_precipitation): #dus and rainfall for each scenario 
    scenar+=1 #scenario counter
    lpreci=[] #precipitation list in each year
    for years_preci in precipitation: #list the values for each year over the lifespan
        lpreci.extend(years_preci.split(','))
    lr=len(lpreci) #precipitation list size in the year
    txtarifftotal=txcoststotal=txdiscounttotal=1 #initial tariff, costs and discount rate values 
    txdiscount=td #td is the discount rate per month (input data)

    for (tx1,tx2,tx3) in zip (tariff, costs, rdiscount): #dus and rainfall for each year of scenario 
        txtarifftotal*=Decimal(tx1) #updates the water tariff increase value
        txcoststotal*=Decimal(tx2) #updates the cost rate increase value
        txdiscount*=Decimal(tx3) #updates the discount rate increase amount in the year
    txdiscounttotal*=((1+float(txdiscount/100))**12) #calculates the total discount rate increase amount in the year
    tttariff=(((float(txtarifftotal))**(1/float(lifespan)))-1)*100 #updates the water tariff value
    ttcosts=(((float(txcoststotal))**(1/float(lifespan)))-1)*100 #updates the cost rate value
    ttdiscount=(((float(1+(((float(txdiscounttotal))**(1/float(lifespan*12)))-1))**12)-1)*100) #updates the discount rate value in the month
    preciptotal=0 #total precipitation in the year
    for dataprecip in lpreci: #precipitation in each day
        preciptotal+=(Decimal(dataprecip)/lifespan) #add precipitation and divide by lifespan
    
    demand=d0
    d1=len(d0) #demand counter starts with qty demand list elements
    de=len(d0) #qty elements demand list
    for item in range (de,lr): #repeat the demand list until it has the same size of precipitation list
        d1+=1
        d2=d1-de-1
        demand.append(Decimal(d0[d2]))  
    ldemand=len(demand) #demand list size
     
    lr=[0] #list of evaluated reservoir volumes
    lsd=[0] #list of satisfied demand for each volume
    lct=[0] #total consumption list
    lve=[0] #extravasated volume list
    lrh=[0] #efficiency list
    lcm=[0] #monthly consumption list
    lrel=[0] #realiability list
    for res in l_volumes: #for a reservoir volume in the volume list
        v0=0 #volume in reservoir in the beginning
        dt=0 #total demand
        ct=0 #total consumption
        qt=0 #total available volume
        drel=0 #days of reliability - that meets the demand completely
        date=date0 #date
        d10=d20=d30=Decimal(0) #consumptions on days 1, 2 and 3
        lcmr=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly consumption list for all volumes of reservoirs

        for (j,k) in zip (lpreci,demand): #sizing for each day of precipitation and demand
            if (Decimal(d10)<Decimal(1)) and (Decimal(d20)<Decimal(1)) and (Decimal(d30)<Decimal(1)):  
                discard_i=discard_ini #if the precipitation on days 1, 2 and 3 is less than 1 mm, discard
            else:
                discard_i=Decimal(0) #if the precipitation on days 1, 2 and 3 is bigger than 1 mm, the discard is 0
            d30=d20
            d20=d10
            d10=j
            q=(Decimal(j)-discard_i)*area*ces/1000 #volume available per day in m³ (precipitation - discard)
            if q<=0: #if q is less than 0 it is zero
                q=0
            vc=min((Decimal(k)/1000),((teta*Decimal(q))+v0)) #volume consumed = demand or vol of the day + vol of the previous day
            if vc>=Decimal(k)/1000: #if demand is consumed, add the days of reliability
                drel+=1
            v=min((q+v0-vc),(res-((1-teta)*vc))) #volume available for the next day
            v0=v
            dt+=(Decimal(k))/1000 #total demand
            ct+=vc #total consumption
            qt+=q #total precipitation volume available
            
            month=date.month #look at the month to calculate rainwater consumption per month            
            lcmr[month-1]+=vc/lifespan #sum the daily consumptions of each month and divide by the qty of years - average monthly consumption
            date=date+timedelta(days=1) #update the date
          
        con_total=sum(lcmr) #average consumption per year of rainwater
        rel=(drel/len(lpreci))*100 #reliability = days with demand fully met/ total days in %
        ve=qt-ct #overflow volume = available - consumption
        rh=(ct/qt)*100 #consumption in relation to available volume in %
        sd=(ct/dt)*100 #consumption in relation to total demand
        
        lr.append(res) #list of reservoirs
        lsd.append(sd) #satisfied demand list - demand met
        lct.append(ct) #total consumption list
        lve.append(ve) #overflow volume list
        lrh.append(rh) #list of consumption efficiency in relation to available volume
        lcm.append(lcmr) #monthly consumption list
        lrel.append(rel) #reliability list
        
    sdi=lsd[0] #first sd of the lsd list
    reseri=lr[0] #first volume of the lr reservoir list
    
    z=ideal=economy=index=0
    for l in range (1,len(lsd)): #calculate the ideal reservoir from the sd
        de=(lsd[l]-sdi)/(lr[l]-reseri) #difference between % of demands met (sd) per m³ of storage
        sdi=lsd[l] #updates the sd
        reseri=lr[l] #update volume
        try:
            if de<=efficiency: #difference less than the desired efficiency
                ideal=lr[l] #ideal volume
                index=lr.index(lr[l]) #optimal reservoir position in the reservoir list
                economy=lsd[l] #savings provided by the ideal reservoir
                z=1 #when z=1 there is an ideal reservoir
                break
        except:
            print(f'It is not possible to get a storage volume for this water-saving potential!')
    
#ECONOMIC EVALUATION
    del (lr[0]) #exclude reservoir 0 because it does not have the system
    del (lcm[0]) #exclude reservoir 0 because it does not have the system
    del (lct[0]) #exclude reservoir 0 because it does not have the system
    del (lsd[0]) #exclude reservoir 0 because it does not have the system
    del (lve[0]) #exclude reservoir 0 because it does not have the system
    del (lrh[0]) #exclude reservoir 0 because it does not have the system
    del (lrel[0]) #exclude reservoir 0 because it does not have the system
    
    lres=[] #list of evaluated reservoir volumes
    lnpv=[] #npv list
    lbcr=[] #bcr list
    lnpvvolume=[] #npvv list
    for (a,b,vol,costinitial) in zip (lr,lcm,lct,l_costsres): #to assess all storage volumes in each scenario - list of reservoirs, monthly consumption, total consumption and cost of reservoirs
        ano=1
        m=0 #month
        revenues=0 #revenues
        vacum=0 #accumulated volume
        list_du_tariff=[]
        list_du_costs=[]
        list_du_discount=[]
        du_tariff=du_costs=du_discount=t_discount=1
        for taa,tac,tad in zip (tariff,costs,rdiscount): #to assess each year's rates of increase under all scenarios 
     
            yeare=0 
            while yeare<=1: #to change the rate values (taa,tac and tad) each year           
                for c,d in zip (b,l_c): #monthly values of rainwater consumption and water consumption of the building
                    yeare+=1
                    m+=1              
                    w=0
                    e1=e2=0
                    p1=p2=tbasic
                    for f,g in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
                        if d>f: #there are more consumption ranges to consider
                            p1+=(f-e1)*g #volume consumed in the consumption range times the cost/m³ of this range
                        else: #there are no more consumption ranges to consider
                            p1+=(d-e1)*g #volume consumed in the consumption range times the cost/m³ of this range
                            break
                        e1=f #to remove the volume already quantified in the previous track
                    for j,k in zip (l_v,l_t): #consumption ranges and prices for total volume - volume saved with RWHS
                        if tmi=='Y' and int(d-c)<tmin: #considering that the minimum rate only exists in case you have a RWHS
                            p2=tmin*k #d-c is the total consumption of the building - the rainwater saving
                            break
                        else:
                            if (d-c)>j: #there are more consumption ranges to consider
                                p2+=(j-e2)*k #volume consumed in the consumption range times the cost/m³ of this range
                            else: #there are no more consumption ranges to consider
                                p2+=((d-c)-e2)*k #volume consumed in the consumption range times the cost/m³ of this range
                                break
                            e2=j #to remove the volume already quantified in the previous track
                     
                    if m==1 or m==2 or m==4 or m==5 or m==7 or m==8 or m==10 or m==11: #to calculate costs for different months
                        costm=cmonthlyf+cmonthlyv*c #months that only have a monthly cost
                    elif m==3 or m==9: #months with monthly and quarterly cost
                        costm=cmonthlyf+cmonthlyv*c+cquarterly
                    elif m==6: #month with monthly, quarterly and half-yearly cost
                        costm=cmonthlyf+cmonthlyv*c+cquarterly+csemester
                    elif m==12: #month with monthly, quarterly, semi-annual and annual cost
                        costm=cmonthlyf+cmonthlyv*c+cquarterly+csemester+cannualf+cannualv*costinitial
                    
                    revenues=(p1-p2)*(1+(esg/100))*Decimal(du_tariff)*Decimal(taa) #price*(% sewage)*(annual water increase rate)
                    t_discount*=1+((td/100)*Decimal(du_discount)*Decimal(tad)) #discount rate - sum of all months
                    vp=(revenues-(costm*Decimal(du_costs)*Decimal(tac)))/(t_discount) #present value
                    vacum+=vp
                    
                    if m==12: #each year applies increases in water, costs and discount rate
                        m-=12
                        ano+=1
                        du_tariff*=Decimal(taa) #value of water increase each year
                        du_costs*=Decimal(tac) #value of costs increase each year
                        du_discount*=Decimal(tad) #value of discount rate increase each year

                list_du_tariff.append(Decimal(du_tariff)*Decimal(taa)) #value of water increase with time
                list_du_costs.append(Decimal(du_costs)*Decimal(tac)) #value of costs increase over time
                list_du_discount.append(t_discount) #value of the increase in the discount rate over time
                    
        npv=vacum-costinitial
        bcr=vacum/costinitial
        npvvolume=(npv)/(vol) #NPV divided by total volume saved
        lres.append(a) #list of reservoirs
        lnpv.append(npv) #NPV list
        lbcr.append(bcr) #BCR list
        lnpvvolume.append(npvvolume) #NPVV list
        
##To save the results to a table
    maxnpv=Decimal(max(lnpv)) #max NPV for all volumes in NPV list
    posicaonpvmax=lnpv.index(maxnpv) #NPV max position in the list 
    rnpv=lres[posicaonpvmax] #reservoir with max NPV
    satismaxnpv=lsd[posicaonpvmax]
    consumptionmaxnpv=lct[posicaonpvmax]/lifespan   
    vextramaxnpv=lve[posicaonpvmax]/lifespan 
    npvvmaxnpv=lnpvvolume[posicaonpvmax]
    bcrmaxnpv=lbcr[posicaonpvmax]
    relmaxnpv=lrel[posicaonpvmax]
    rhmaxnpv=lrh[posicaonpvmax]    
    
    maxbcr=max(lbcr) #max BCR for all volumes in BCR list
    posicaobcrmax=lbcr.index(maxbcr) #BCR max position in the list 
    rbcr=lres[posicaobcrmax] #reservoir with max BCR    
    satismaxbcr=lsd[posicaobcrmax]
    consumptionmaxbcr=lct[posicaobcrmax]/lifespan 
    vextramaxbcr=lve[posicaobcrmax]/lifespan 
    npvmaxbcr=lnpv[posicaobcrmax]
    npvvmaxbcr=lnpvvolume[posicaobcrmax]   
    relmaxbcr=lrel[posicaobcrmax]
    rhmaxbcr=lrh[posicaobcrmax]

    maxnpvv=max(lnpvvolume) #max NPVV for all volumes in NPVV list
    posicaonpvvmax=lnpvvolume.index(maxnpvv) #NPVV max position in list
    rmaxnpvv=lres[posicaonpvvmax] #reservoir with the highest NPVV
    satismaxnpvv=lsd[posicaonpvvmax]
    consumptionmaxnpvv=lct[posicaonpvvmax]/lifespan     
    vextramaxnpvv=lve[posicaonpvvmax]/lifespan 
    npvmaxnpvv=lnpv[posicaonpvvmax]
    bcrmaxnpvv=lbcr[posicaonpvvmax]
    relmaxnpvv=lrel[posicaonpvvmax]
    rhmaxnpvv=lrh[posicaonpvvmax] 

    maxrel=max(lrel) #max REL for all volumes in the REL list
    posicaomaxrel=lrel.index(maxrel) #max REL position for all volumes in REL list
    rmaxrel=lres[posicaomaxrel] #reservoir that presents a system with greater REL
    satismaxrel=lsd[posicaomaxrel]
    consumptionmaxrel=lct[posicaomaxrel]/lifespan     
    vextramaxrel=lve[posicaomaxrel]/lifespan 
    npvmaxrel=lnpv[posicaomaxrel]
    npvvmaxrel=lnpvvolume[posicaomaxrel] 
    bcrmaxrel=lbcr[posicaomaxrel]
    rhmaxrel=lrh[posicaomaxrel] 
    
    satismax=max(lsd) #max SD for all volumes in the SD list
    posicaomaxe=lsd.index(satismax) #max SD position for all volumes in the SD list
    rmaxe=lres[posicaomaxe] #reservoir that presents a system with greater SD
    consumptionmaxe=lct[posicaomaxe]/lifespan     
    vextramaxe=lve[posicaomaxe]/lifespan 
    npvmaxe=lnpv[posicaomaxe]
    npvvmaxe=lnpvvolume[posicaomaxe] 
    bcrmaxe=lbcr[posicaomaxe]
    relmaxe=lrel[posicaomaxe]
    rhmaxe=lrh[posicaomaxe]     
    
    rhmax=max(lrh) #max RH for all volumes in the RH list 
    posicaomaxrh=lrh.index(rhmax) #max RH position for all volumes in RH list
    rmaxrh=lres[posicaomaxrh] #reservoir that presents a system with greater RH
    consumptionmaxrh=lct[posicaomaxrh]/lifespan     
    vextramaxrh=lve[posicaomaxrh]/lifespan 
    npvmaxrh=lnpv[posicaomaxrh]
    npvvmaxrh=lnpvvolume[posicaomaxrh] 
    bcrmaxrh=lbcr[posicaomaxrh]
    relmaxrh=lrel[posicaomaxrh]
    satismaxrh=lsd[posicaomaxrh]      
    
    if z==0:
        costideal=lct[index-1]=lve[index-1]=npv_ideal=npvv_ideal=bcr_ideal=relideal=rhideal=0 #if there is no ideal volume consider 0
    else:
        costideal=l_costsres[index-1] #ideal reservoir cost
        npv_ideal=lnpv[index-1] #ideal reservoir NPV
        npvv_ideal=lnpvvolume[index-1] #ideal reservoir NPVV
        bcr_ideal=lbcr[index-1] #ideal reservoir BCR
        relideal=lrel[index-1] #ideal reservoir REL
        rhideal=lrh[index-1] #ideal reservoir RH
    demandyear=demand[0]*365/1000

#SAVE RESULTS TO THE FILE
    with open ('results_IP_demand3_area200.csv','a') as file: #save each scenario results to file
        file.write(str(scenar)+','+str(preciptotal)+','+str(tx1)+','+str(tx2)+','+str(tx3)+','+str(area)+\
                      ','+str(demandyear)+','+str(ideal)+','+str(economy)+','+str((lct[index-1])/lifespan)+','+str((lve[index-1])/lifespan)+\
                      ','+str(npv_ideal)+','+str(npvv_ideal)+','+str(bcr_ideal)+','+str(relideal)+','+str(rhideal)+','+str(costideal)+\
                      ','+str(rnpv)+','+str(satismaxnpv)+','+str(consumptionmaxnpv)+','+str(vextramaxnpv)+','+str(maxnpv)+\
                      ','+str(npvvmaxnpv)+','+str(bcrmaxnpv)+','+str(relmaxnpv)+','+str(rhmaxnpv)+\
                      ','+str(rbcr)+','+str(satismaxbcr)+','+str(consumptionmaxbcr)+','+str(vextramaxbcr)+','+str(npvmaxbcr)+\
                      ','+str(npvvmaxbcr)+','+str(maxbcr)+','+str(relmaxbcr)+','+str(rhmaxbcr)+\
                      ','+str(rmaxnpvv)+','+str(satismaxnpvv)+','+str(consumptionmaxnpvv)+','+str(vextramaxnpvv)+','+str(npvmaxnpvv)+\
                      ','+str(maxnpvv)+','+str(bcrmaxnpvv)+','+str(relmaxnpvv)+','+str(rhmaxnpvv)+\
                      ','+str(rmaxrel)+','+str(satismaxrel)+','+str(consumptionmaxrel)+','+str(vextramaxrel)+','+str(npvmaxrel)+\
                      ','+str(npvvmaxrel)+','+str(bcrmaxrel)+','+str(maxrel)+','+str(rhmaxrel)+\
                      ','+str(rmaxe)+','+str(satismax)+','+str(consumptionmaxe)+','+str(vextramaxe)+','+str(npvmaxe)+\
                      ','+str(npvvmaxe)+','+str(bcrmaxe)+','+str(relmaxe)+','+str(rhmaxe)+\
                      ','+str(rmaxrh)+','+str(satismaxrh)+','+str(consumptionmaxrh)+','+str(vextramaxrh)+','+str(npvmaxrh)+\
                      ','+str(npvvmaxrh)+','+str(bcrmaxrh)+','+str(relmaxrh)+','+str(rhmax)+'\n')
print(f'END!!')

END!!
